In [1]:
import pandas as pd
import re

import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
from matplotlib import pyplot as plt

%matplotlib inline
import seaborn as sns
import eli5
import numpy as np
import nltk

nltk.download("stopwords")
from nltk.corpus import stopwords

russian_stopwords = stopwords.words("russian")

[nltk_data] Downloading package stopwords to /home/andrey/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Модуль с загрузкой документа и извлечением необходимых данных

In [2]:
import os
import docx
import re
import pickle

In [3]:
def get_text(doc):
    fullText = []
    for para in doc.paragraphs:
        fullText.append(para.text.strip())
    fullText = [elem for elem in fullText if len(elem) > 0]
    return fullText

In [4]:
def get_first_index(fullText):
    ind_start = 0
    for i in range(len(fullText) - 1):
        if ("правила" in fullText[i].lower()) and (
            "предоставления" in fullText[i + 1].lower()
        ):
            ind_start = i
            break
    return ind_start

In [5]:
def get_end_index(fullText):
    end_start = len(fullText)
    for i in range(ind_start, len(fullText) - 1):
        if (
            (
                fullText[i].lower().startswith("утвержден")
                & fullText[i + 1].lower().startswith("постановлением правительства")
            )
            | (
                ("приложение №" in fullText[i].lower())
                & ("предоставления" in fullText[i + 1].lower())
            )
            | (
                ("приложение №" in fullText[i].lower())
                & ("предоставления" in fullText[i].lower())
            )
            | (
                ("приложение N" in fullText[i].lower())
                & ("предоставления" in fullText[i + 1].lower())
            )
            | (
                ("приложение" in fullText[i].lower())
                & ("к правилам предоставления" in fullText[i + 1].lower())
            )
        ):
            if (i < end_start - 6) & ("правила" in fullText[i + 4].lower()) and (
                "предоставления" in fullText[i + 5].lower()
            ):
                continue
            end_start = i
            break
    return end_start

In [6]:
path_docs = "../data/test_real/"

In [7]:
interest_files = [file for file in os.listdir(path_docs) if file.endswith("docx")]
sorted(interest_files)

['1.docx',
 '10.docx',
 '2.docx',
 '3.docx',
 '4.docx',
 '5.docx',
 '6.docx',
 '7.docx',
 '8.docx',
 '9.docx']

In [8]:
sample_sub = pd.read_csv("../data/test_real/description/Название_команды.csv", sep=";")
sample_sub

,file_id,id,class
0,1,1,1
1,1,2,1
2,1,3,1
3,1,4,1
4,1,5,1
...,...,...,...
847,10,67,1
848,10,68,1
849,10,69,1
850,10,70,1


In [9]:
sample_sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 852 entries, 0 to 851
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   file_id  852 non-null    int64
 1   id       852 non-null    int64
 2   class    852 non-null    int64
dtypes: int64(3)
memory usage: 20.1 KB


### определяем текущую модель

In [10]:
# для WEB
filename = "../models/tfidf_model.pickle"
filename = "../models/logreg_model.pickle"
# filename = "../models/ensamble_model.pickle"

In [11]:
# для теста решения
filename = "../models/tfidf_model_all_data.pickle"
text_transformer = pickle.load(open(filename, "rb"))

filename = "../models/logreg_model_all_data.pickle"

model = pickle.load(open(filename, "rb"))

/home/andrey/miniconda3/lib/python3.7/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator TfidfTransformer from version 1.0.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/andrey/miniconda3/lib/python3.7/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 1.0.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/andrey/miniconda3/lib/python3.7/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator LogisticRegression from version 1.0.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


### поехали

In [12]:
df_submission = []
for cur_doc in sorted(interest_files):
    filename = path_docs + cur_doc
    file_id = int(cur_doc.split(".")[0])
    doc = docx.Document(filename)
    fullText = get_text(doc)
    # сегментируем, где у нас потенциально есть классы
    ind_start = get_first_index(fullText)
    end_start = get_end_index(fullText)

    target_data = []
    activate_flag = False
    real_start_real = ind_start
    for num_el, elem in enumerate(fullText[ind_start:end_start]):
        if not activate_flag:
            if len(re.findall("\d+\. ", elem)) > 0:
                real_start_real += num_el
                activate_flag = True
                target_data.append(elem.replace("\xa0", " "))
        else:
            target_data.append(elem.replace("\xa0", " "))
    # извлекаем текст и id
    texts = []
    ids = []
    for num, elem in enumerate(target_data):
        if len(re.findall("{\d+", elem)) > 0:
            splits = re.split("\d+}", elem)
            cur_ids = re.findall("\d+}", elem)
            for i in range(1, len(splits), 2):
                texts.append(splits[i])
            for i in range(1, len(cur_ids), 2):
                ids.append(int(re.findall("\d+", cur_ids[i])[0]))

    df1 = pd.DataFrame({"texts": texts, "id": ids})

    # работа для модели
    ref_text = [re.sub("[\W\d]+", " ", x.lower()) for x in df1["texts"]]
    X_text = text_transformer.transform(ref_text)
    y_pred = model.predict(X_text)
    #     y_proba = model.predict_proba(X_text)
    df1["class"] = y_pred
    df1["file_id"] = file_id
    df_submission.append(df1)

In [13]:
df_submission = pd.concat(df_submission)

In [14]:
df_submission = df_submission[["file_id", "id", "class", "texts"]]

In [15]:
df_submission.sort_values(["file_id", "id"], inplace=True)

In [16]:
df_submission

,file_id,id,class,texts
0,1,1,2,"Настоящие Правила устанавливают цели, условия ..."
1,1,2,2,Субсидия предоставляется получателю субсидии ...
2,1,3,2,"Субсидия предоставляется в рамках ""Информацио..."
3,1,4,3,Субсидия предоставляется в пределах лимитов бю...
4,1,5,7,Сведения (информация) о субсидии размещаются ...
...,...,...,...,...
66,10,67,38,на основании представления и (или) предписания...
67,10,68,38,В случае недостижения организацией значений ре...
68,10,69,22,Министерство транспорта Российской Федерации н...
69,10,70,38,Срок исполнения организацией требования не дол...


In [17]:
df_submission.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 852 entries, 0 to 70
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   file_id  852 non-null    int64 
 1   id       852 non-null    int64 
 2   class    852 non-null    int64 
 3   texts    852 non-null    object
dtypes: int64(3), object(1)
memory usage: 33.3+ KB


In [18]:
sample_sub

,file_id,id,class
0,1,1,1
1,1,2,1
2,1,3,1
3,1,4,1
4,1,5,1
...,...,...,...
847,10,67,1
848,10,68,1
849,10,69,1
850,10,70,1


In [19]:
sample_sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 852 entries, 0 to 851
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   file_id  852 non-null    int64
 1   id       852 non-null    int64
 2   class    852 non-null    int64
dtypes: int64(3)
memory usage: 20.1 KB


In [20]:
df_submission[sample_sub.columns].to_csv("SibDS.csv", sep=";", index=False)